## The notebook will show you why get_dummies is not ideal for deployable models  

This article goes into more depth about OHE for multiple colums

https://vitalflux.com/one-hot-encoding-concepts-python-code-examples/

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Create  dataframe with categorical column
# We are going to predict Age based on Animal

In [3]:
data = {'Animal': ['Dog', 'Fish', 'Cat', 'Horse'], 'Age': [20, 21, 19, 18]}  


In [4]:
df = pd.DataFrame(data)  


In [5]:
#Artificailly make the df larger so we have some decent size data to work with

In [6]:
for x in range(10):
    df = pd.concat([df,df])

In [7]:
df.shape

(4096, 2)

In [8]:
# Use get_dummies to encode the categorical
# This looks to be working fine and it is for training and scoring the model

In [9]:
df = pd.get_dummies(df)

In [10]:
df.head()

,Age,Animal_Cat,Animal_Dog,Animal_Fish,Animal_Horse
0,20,0,1,0,0
1,21,0,0,1,0
2,19,1,0,0,0
3,18,0,0,0,1
0,20,0,1,0,0


In [11]:
y = df.pop('Age')

In [12]:
X_train,X_test,y_train,y_test = train_test_split(df,y)

In [16]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [18]:
model.score(X_test,y_test)

1.0

### At this point you have trained a model on training data and scored the model on test data.  You deploy your model to production and you are ready to receive new records on which to make a prediction

In [ ]:
# New data like the record below is sent in.

In [14]:
new_data = {'Animal': ['Dog']}  


In [19]:
# Like above, you convert you new record into a dataframe and dummy it to get rid of the categorical variable

In [15]:
new_data
new_data = pd.DataFrame(new_data)
new_data = pd.get_dummies(new_data)

In [16]:
new_data

,Animal_Dog
0,1


In [20]:
# You now want to feed it into your trained model and it breaks because your model was expecting your data to have
# a different shape but the get_dummies has no "memory" of all the the possible categoricals you trained on

In [17]:
model.predict(new_data)

/home/jeremy/anaconda3/lib/python3.8/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Animal_Cat
- Animal_Fish
- Animal_Horse

  warnings.warn(message, FutureWarning)


ValueError: X has 1 features, but RandomForestRegressor is expecting 4 features as input.

In [ ]:
# The solution to this problem is a One Hot Encoder object

In [ ]:
# Generate the same dataset

In [66]:
df = pd.DataFrame(data)  
for x in range(10):
    df = pd.concat([df,df])
df.shape

(4096, 2)

In [67]:
#Split your data before encoding to prevent any data leakage into test set

In [68]:
y = df.pop('Age')
X_train,X_test,y_train,y_test = train_test_split(df,y)

In [69]:
OHE = OneHotEncoder()

In [70]:
# creates a dataframe with the encoded variables.  

In [72]:
encoded_train = pd.DataFrame(OHE.fit_transform(X_train[['Animal']]).toarray())
encoded_train.columns = OHE.get_feature_names_out()

In [73]:
# Train model on encoded data

In [74]:
model = RandomForestRegressor()
model.fit(encoded_train, y_train)

RandomForestRegressor()

In [75]:
encoded_test = pd.DataFrame(OHE.transform(X_test[['Animal']]).toarray())
encoded_test.columns = OHE.get_feature_names_out()

In [76]:
# Score your model

In [77]:
model.score(encoded_test,y_test)

1.0

### Now when you pass in a new record, you apply the trained OHE to the record and it creates a df of the correct shape

In [83]:
new_data = {'Animal': ['Dog']}  
new_data = pd.DataFrame(new_data)

In [84]:
new_data

,Animal
0,Dog


In [85]:
encoded_new_data = pd.DataFrame(OHE.transform(new_data[['Animal']]).toarray())

In [89]:
encoded_new_data.columns = OHE.get_feature_names_out()


In [90]:
encoded_new_data

,Animal_Cat,Animal_Dog,Animal_Fish,Animal_Horse
0,0.0,1.0,0.0,0.0


In [91]:
# When ou do your predict, you get the right answer and not an error.

In [92]:
model.predict(encoded_new_data)

array([20.])

In [93]:
# For deployment, you would deploy the trained OHE object and the trained Regression model object